In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
import random
from ipywidgets import Button, Output, VBox
from IPython.display import display, clear_output

# ----------------------------
# Adaline Class
# ----------------------------
class Adaline:
    def __init__(self, learning_rate=0.00001, n_iterations=20):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.errors_ = []

    def fit(self, X, y):
        self.weights = np.random.normal(0, 0.01, size=(1 + X.shape[1]))
        for epoch in range(self.n_iterations):
            net_input = self.net_input(X)
            output = self.activation(net_input)
            errors = y - output
            self.weights[1:] += self.learning_rate * (X.T.dot(errors) - 0.01 * self.weights[1:])
            self.weights[0] += self.learning_rate * errors.sum()
            self.weights = np.clip(self.weights, -1e3, 1e3)
            mse = np.mean(errors**2)
            self.errors_.append(mse)
            print(f"Epoch {epoch + 1}: Mean Error = {mse:.5f}, Weights Mean = {np.mean(self.weights):.5f}")
        return self

    def net_input(self, X):
        return np.dot(X, self.weights[1:]) + self.weights[0]

    def activation(self, X):
        return X

    def predict(self, X):
        return np.where(self.activation(self.net_input(X)) >= 0.5, 1, 0)

# ----------------------------
# Madaline Class
# ----------------------------
class Madaline:
    def __init__(self, n_adalines=3, learning_rate=0.00001, n_iterations=20):
        self.n_adalines = n_adalines
        self.adalines = [Adaline(learning_rate, n_iterations) for _ in range(n_adalines)]

    def fit(self, X, y):
        feature_split = np.array_split(X, self.n_adalines, axis=1)
        for i, adaline in enumerate(self.adalines):
            adaline.fit(feature_split[i], y)
        return self

    def predict(self, X):
        feature_split = np.array_split(X, self.n_adalines, axis=1)
        predictions = np.array([adaline.predict(split) for adaline, split in zip(self.adalines, feature_split)])
        return np.where(predictions.mean(axis=0) >= 0.5, 1, 0)

# ----------------------------
# Shape Generation
# ----------------------------
def generate_shape(shape_type, image_size=64, noise_level=0.1):
    """Generate a simple shape image with noise."""
    image = np.zeros((image_size, image_size), dtype=np.uint8)
    center = (image_size // 2, image_size // 2)
    size = int(image_size * 0.3)

    if shape_type == 'circle':
        cv2.circle(image, center, size, 255, -1)
    elif shape_type == 'square':
        top_left = (center[0] - size, center[1] - size)
        bottom_right = (center[0] + size, center[1] + size)
        cv2.rectangle(image, top_left, bottom_right, 255, -1)
    elif shape_type == 'triangle':
        points = np.array([
            [center[0], center[1] - size],
            [center[0] - size, center[1] + size],
            [center[0] + size, center[1] + size]
        ], np.int32)
        cv2.fillPoly(image, [points], 255)
    elif shape_type == 'pentagon':
        points = np.array([
            [center[0], center[1] - size],
            [center[0] - size, center[1] - size // 2],
            [center[0] - size // 2, center[1] + size],
            [center[0] + size // 2, center[1] + size],
            [center[0] + size, center[1] - size // 2]
        ], np.int32)
        cv2.fillPoly(image, [points], 255)
    elif shape_type == 'hexagon':
        points = np.array([
            [center[0] - size, center[1]],
            [center[0] - size // 2, center[1] - size],
            [center[0] + size // 2, center[1] - size],
            [center[0] + size, center[1]],
            [center[0] + size // 2, center[1] + size],
            [center[0] - size // 2, center[1] + size]
        ], np.int32)
        cv2.fillPoly(image, [points], 255)

    noise = np.random.normal(0, noise_level, image.shape)
    noisy_image = image + noise * 255
    return np.clip(noisy_image, 0, 255).astype(np.uint8)

def create_dataset(n_samples=500):
    """Create a dataset of shapes."""
    X = []
    y = []
    shapes = ['circle', 'square', 'triangle', 'pentagon', 'hexagon']
    for _ in range(n_samples):
        shape = random.choice(shapes)
        image = generate_shape(shape)
        X.append(image.flatten())
        y.append([1 if shape == s else 0 for s in shapes])
    return np.array(X), np.array(y)

# ----------------------------
# Helper Functions
# ----------------------------
def predict_shape(image, models):
    """Predict the shape using trained models."""
    image_scaled = scaler.transform([image.flatten()])
    predictions = {}
    for shape, model in models.items():
        adaline_pred = model['adaline'].predict(image_scaled)[0]
        madaline_pred = model['madaline'].predict(image_scaled)[0]
        predictions[shape] = (adaline_pred + madaline_pred) / 2
    return max(predictions.items(), key=lambda x: x[1])[0]

def calculate_accuracy():
    """Calculate the accuracy of the models."""
    correct = 0
    total = len(X_test)
    for i in range(total):
        actual_shape = shapes[np.argmax(y_test[i])]
        predicted_shape = predict_shape(X_test[i].reshape(64, 64), shape_models)
        if actual_shape == predicted_shape:
            correct += 1
    return correct / total

# ----------------------------
# Main Program
# ----------------------------
print("Initializing Shape Recognition System...")
X, y = create_dataset(500)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

shapes = ['circle', 'square', 'triangle', 'pentagon', 'hexagon']
shape_models = {}

for i, shape in enumerate(shapes):
    print(f"Training models for {shape}...")
    adaline = Adaline(learning_rate=0.00001, n_iterations=20)
    adaline.fit(X_train_scaled, y_train[:, i])
    madaline = Madaline(n_adalines=3, learning_rate=0.00001, n_iterations=20)
    madaline.fit(X_train_scaled, y_train[:, i])
    shape_models[shape] = {'adaline': adaline, 'madaline': madaline}

accuracy = calculate_accuracy()
print(f"\nModel Accuracy: {accuracy:.2%}")

# ----------------------------
# Interactive Buttons for Colab
# ----------------------------
output = Output()

def generate_shape_and_test(_):
    with output:
        clear_output(wait=True)
        random_shape = random.choice(shapes)
        test_image = generate_shape(random_shape)
        predicted_shape = predict_shape(test_image, shape_models)
        plt.imshow(test_image, cmap='gray')
        plt.title(f'Generated: {random_shape}\nPredicted: {predicted_shape}')
        plt.axis('off')
        plt.show()

def view_accuracy_and_plot(_):
    with output:
        clear_output(wait=True)
        print(f"\nCurrent Model Accuracy: {accuracy:.2%}")
        print("\nPlotting Error Progression for Each Shape...\n")

        # Plot error progression for each shape
        plt.figure(figsize=(15, 10))
        for i, shape in enumerate(shapes):
            adaline_errors = shape_models[shape]['adaline'].errors_
            madaline_errors = []
            for adaline in shape_models[shape]['madaline'].adalines:
                madaline_errors.append(adaline.errors_)

            # Adaline errors
            plt.subplot(len(shapes), 2, i * 2 + 1)
            plt.plot(adaline_errors, label=f'Adaline - {shape.capitalize()}')
            plt.title(f'{shape.capitalize()} - Adaline Error')
            plt.xlabel('Epochs')
            plt.ylabel('Mean Squared Error')
            plt.legend()

            # Madaline errors
            plt.subplot(len(shapes), 2, i * 2 + 2)
            for j, errors in enumerate(madaline_errors):
                plt.plot(errors, label=f'Madaline Adaline-{j + 1}')
            plt.title(f'{shape.capitalize()} - Madaline Error')
            plt.xlabel('Epochs')
            plt.ylabel('Mean Squared Error')
            plt.legend()

        plt.tight_layout()
        plt.show()

def exit_program(_):
    with output:
        clear_output(wait=True)
        print("Thank you for using the Shape Recognition System!")

button1 = Button(description="Generate and Test Random Shape")
button1.on_click(generate_shape_and_test)

button2 = Button(description="View Model Accuracy and Errors")
button2.on_click(view_accuracy_and_plot)

button3 = Button(description="Exit")
button3.on_click(exit_program)

display(VBox([button1, button2, button3]), output)


Initializing Shape Recognition System...
Training models for circle...
Epoch 1: Mean Error = 0.55090, Weights Mean = -0.00014
Epoch 2: Mean Error = 0.42201, Weights Mean = -0.00019
Epoch 3: Mean Error = 0.35623, Weights Mean = -0.00015
Epoch 4: Mean Error = 0.31490, Weights Mean = -0.00018
Epoch 5: Mean Error = 0.28615, Weights Mean = -0.00016
Epoch 6: Mean Error = 0.26434, Weights Mean = -0.00017
Epoch 7: Mean Error = 0.24660, Weights Mean = -0.00016
Epoch 8: Mean Error = 0.23144, Weights Mean = -0.00017
Epoch 9: Mean Error = 0.21805, Weights Mean = -0.00016
Epoch 10: Mean Error = 0.20598, Weights Mean = -0.00016
Epoch 11: Mean Error = 0.19496, Weights Mean = -0.00016
Epoch 12: Mean Error = 0.18482, Weights Mean = -0.00016
Epoch 13: Mean Error = 0.17545, Weights Mean = -0.00015
Epoch 14: Mean Error = 0.16676, Weights Mean = -0.00016
Epoch 15: Mean Error = 0.15869, Weights Mean = -0.00015
Epoch 16: Mean Error = 0.15118, Weights Mean = -0.00015
Epoch 17: Mean Error = 0.14418, Weights Me

Output()